In [9]:
# initialize objects and classes 
from collections import defaultdict

# this is the stock market 
class stockMarket:
    def __init__(self):
        self.buyOrders = {}
        self.sellOrders = {}
        self.completedOrders = {}
    def buyOrder(self, userInput):
        # remember limit order and market order
        # O(n) time where n is the number of market orders on the stock requested

        
        typeOfOrder = self.determineTypeOfOrder(userInput)

        if typeOfOrder == None:
            print("Invalid Input! typeOfOrder")
            return 

        # print("typeOfOrder", typeOfOrder)
        
        stockName = self.determineStockName(userInput)

        if stockName == None:
            print("Invalid Input! stockName")
            return 
        # print("stockName", stockName)
        numberOfOrdersPlaced = self.determineNumbersPlaced(typeOfOrder, userInput)

        # print("Branching here", typeOfOrder)
        if typeOfOrder == "MKT":
            # print("MKT BRANCH TAKEN!")
            if stockName not in self.sellOrders:
                # then just add to my buyOrders deque
                newNode = orderNode(numberOfOrdersPlaced, -1, typeOfOrder)
                if stockName not in self.buyOrders: 
                    self.buyOrders[stockName] = DoublyLinkedList()
                    
                    # here I assume the the most recent order will be resolved last. 
                    self.buyOrders[stockName].insertAtPosition(1, newNode)

                # if its current buy orders already exist, just add to the end of the queue
                else:
                    self.insertAtCorrectBuyLMTPosition(newNode, stockName) # the same, can be resued
            else:
                # if it is a market order and there are sell orders available
                startOfSellOrder = self.sellOrders[stockName].head
                askingPrice = startOfSellOrder.price
                if startOfSellOrder.numberOfStocks >= numberOfOrdersPlaced:
                    # that means this order can be completed 
                    if stockName in self.completedOrders:

                        self.completedOrders[stockName].append(["MKT","BUY", askingPrice, numberOfOrdersPlaced])
                    else:
                        self.completedOrders[stockName] = [["MKT" , "BUY", askingPrice, numberOfOrdersPlaced]]
                    sellOrderRemaining = startOfSellOrder.numberOfStocks - numberOfOrdersPlaced
                    startOfSellOrder.numberOfStocks = sellOrderRemaining
                    if startOfSellOrder.numberOfStocks == 0:
                        self.completedOrders[stockName].append([startOfSellOrder.typeOfOrder, "SELL", startOfSellOrder.price, startOfSellOrder.initialNumberOfOrders])
                        self.sellOrders[stockName].remove(startOfSellOrder)
                else: 
                    remainingOrders = numberOfOrdersPlaced
                    while remainingOrders > 0 and startOfSellOrder:
                        if remainingOrders > startOfSellOrder.numberOfStocks:
                            nodeToRemove = startOfSellOrder
                            if stockName in self.completedOrders:
                                self.completedOrders[stockName].append([nodeToRemove.typeOfOrder, "SELL", nodeToRemove.price, nodeToRemove.initialNumberOfOrders])
                            else:
                                self.completedOrders[stockName]=[[nodeToRemove.typeOfOrder, "SELL", nodeToRemove.price, nodeToRemove.initialNumberOfOrders]]
                            remainingOrders -= startOfSellOrder.numberOfStocks
                            startOfSellOrder = startOfSellOrder.next
                            self.sellOrders[stockName].remove(nodeToRemove)
                        else:
                            initial = remainingOrders 
                            remainingOrders -= startOfSellOrder.numberOfStocks
                            startOfSellOrder.numberOfStocks -= initial 
                            if startOfSellOrder.numberOfStocks == 0:
                                # resolve 
                                nodeToRemove = startOfSellOrder 
                                self.completedOrders[stockName] = [nodeToRemove.typeOfOrder, "SELL", nodeToRemove.price, nodeToRemove.initialNumberOfOrders]
                                self.sellOrders[stockName].remove(nodeToRemove)

                            
                            
                    # if numberOfOrdersPlaced has remaining, just add to buyorders
                    if remainingOrders > 0:
                        newNode = orderNode(numberOfOrdersPlaced, -1, typeOfOrder)
                        newNode.numberOfStocks = remainingOrders
                        if stockName not in self.buyOrders:
                            self.buyOrders[stockName] = DoublyLinkedList()
                            self.buyOrders[stockName].insertAtPosition(1, newNode)
                        else:
                            self.insertAtCorrectBuyLMTPosition(newNode, stockName)
                    else:
                        self.completedOrders[stockName].append([typeOfOrder, "BUY", askingPrice,numberOfOrdersPlaced])



        elif typeOfOrder == "LMT": # it is a limit order here 
            # print("LMT BRANCH TAKEN")
            buyingPrice = self.determineBuyingPrice(userInput)
            # print(buyingPrice, "BuyingPrice")
            # print(userInput, "look here")
            newNode = orderNode(numberOfOrdersPlaced, buyingPrice, typeOfOrder)
            if stockName not in self.sellOrders:
                # find suitable position to insert buy orders queue 
                
                if stockName not in self.buyOrders:
                    self.buyOrders[stockName] = DoublyLinkedList()
                    self.buyOrders[stockName].insertAtPosition(1, newNode)
                    
                else: # it is already in buyOrders, insert at correct price position
                    self.insertAtCorrectBuyLMTPosition(newNode, stockName)
            else: # try to resolve the order 
                startOfSellOrder = self.sellOrders[stockName].head
                askingPrice = startOfSellOrder.price if startOfSellOrder.price != -1 else buyingPrice

                remainingOrders = numberOfOrdersPlaced
                while startOfSellOrder and remainingOrders > 0:

                    if (startOfSellOrder.numberOfStocks >= remainingOrders and startOfSellOrder.price == -1) or (startOfSellOrder.numberOfStocks >= remainingOrders and startOfSellOrder.price <= buyingPrice):
                        
                        initial = remainingOrders 
                        remainingOrders -= startOfSellOrder.numberOfStocks
                        startOfSellOrder.numberOfStocks -= initial
                        if stockName in self.completedOrders:
                            self.completedOrders[stockName].append(["LMT", "BUY", askingPrice, numberOfOrdersPlaced])
                            
                        else:
                            self.completedOrders[stockName] = [["LMT", "BUY", askingPrice, numberOfOrdersPlaced]]
                        if startOfSellOrder.numberOfStocks == 0:
                            # check if the sell order is a lmt or mkt 
                            self.sellOrders[stockName].remove(startOfSellOrder)
                            sellOrderType = "LMT"
                            if startOfSellOrder.price == -1: 
                                sellOrderType = "MKT"
                            self.completedOrders[stockName].append([sellOrderType, "SELL", askingPrice, startOfSellOrder.initialNumberOfOrders])
                        break
                    elif (startOfSellOrder.numberOfStocks < remainingOrders and startOfSellOrder.price == -1) or (startOfSellOrder.numberOfStocks < remainingOrders and startOfSellOrder.price <= buyingPrice):
                        # resolve the sell Order 
                        nodeToRemove = startOfSellOrder 
                        remainingOrders -= nodeToRemove.numberOfStocks
                        if stockName in self.completedOrders:
                            self.completedOrders[stockName].append([nodeToRemove.typeOfOrder, "SELL", askingPrice, startOfSellOrder.initialNumberOfOrders])
                        else:
                            self.completedOrders[stockName] = [[nodeToRemove.typeOfOrder, "SELL", askingPrice, startOfSellOrder.initialNumberOfOrders]]
                        startOfSellOrder = startOfSellOrder.next
                        askingPrice = startOfSellOrder.price
                        self.sellOrders[stockName].remove(nodeToRemove)
                    else:
                        startOfSellOrder = startOfSellOrder.next

                if remainingOrders > 0:
                    newNode.numberOfStocks = remainingOrders
                    self.insertAtCorrectBuyLMTPosition(newNode, stockName)
        else: 
            print("nice try")

    
        return 

    def sellOrder(self, userInput):
        # remember limit order and market order \
        # O(n) time where n is the number of market orders on the stock requested
        typeOfOrder = self.determineTypeOfOrder(userInput)

        if typeOfOrder == None:
            print("Invalid Input! typeOfOrder")
            return 

        stockName = self.determineStockName(userInput)

        if stockName == None:
            print("invalid Input! stockname")
            return 

        numberOfOrdersPlaced = self.determineNumbersPlaced(typeOfOrder, userInput)
        if typeOfOrder == "MKT":
            # print("SELL MKT BRANCH TAKEN!")
            if stockName not in self.buyOrders:
                # then just add to sellOrders deque 
                newNode = orderNode(numberOfOrdersPlaced, -1, typeOfOrder)
                if stockName not in self.sellOrders:
                    self.sellOrders[stockName] = DoublyLinkedList()
                    self.sellOrders[stockName].insertAtPosition(1, newNode)
                
                else: # if it already currently exists, add to the end of the queue 
                    self.insertAtCorrectSellLMTPosition(newNode, stockName) # is the same logic
            else: # if there is a sell order and there are buy orders avail 
                startOfBuyOrder = self.buyOrders[stockName].head 
                buyingPrice = startOfBuyOrder.price
                if startOfBuyOrder.numberOfStocks >= numberOfOrdersPlaced:
                    # this means that this order can be completed 
                    if stockName in self.completedOrders:
                        self.completedOrders[stockName].append(["MKT", "SELL", buyingPrice, numberOfOrdersPlaced])
                    else:
                        self.completedOrders[stockName]= [["MKT", "SELL", buyingPrice, numberOfOrdersPlaced]]
                    buyOrderRemaining = startOfBuyOrder.numberOfStocks - numberOfOrdersPlaced
                    startOfBuyOrder.numberOfStocks = buyOrderRemaining
                    if startOfBuyOrder.numberOfStocks == 0:
                        self.completedOrders[stockName].append([startOfBuyOrder.typeOfOrder, "BUY", startOfBuyOrder.price, startOfBuyOrder.initialNumberOfOrders])
                        self.buyOrders[stockName].remove(startOfBuyOrder)
                else:
                    remainingOrders = numberOfOrdersPlaced
                    while remainingOrders > 0 and startOfBuyOrder:
                        if remainingOrders > startOfBuyOrder.numberOfStocks:
                            nodeToRemove = startOfBuyOrder 
                            if stockName in self.completedOrders:
                                self.completedOrders[stockName].append([nodeToRemove.typeOfOrder, "BUY", nodeToRemove.price, nodeToRemove.initialNumberOfOrders])
                            else :
                                self.completedOrders[stockName] = [[nodeToRemove.typeOfOrder, "BUY", nodeToRemove.price, nodeToRemove.initialNumberOfOrders]]
                            remainingOrders -= startOfBuyOrder.numberOfStocks
                            startOfBuyOrder = startOfBuyOrder.next 
                            self.buyOrders[stockName].remove(nodeToRemove)
                        else:
                            initial = remainingOrders
                            remainingOrders -= startOfBuyOrder.numberOfStocks
                            startOfBuyOrder.numberOfStocks -= initial 
                            if startOfBuyOrder == 0:
                                nodeToRemove = startOfBuyOrder 
                                self.completedOrders[stockName].append([nodeToRemove.typeOfOrder, "BUY", nodeToRemove.price, nodeToRemove.initialNumberOfOrders])
                                
                                self.buyOrders[stockName].remove(nodeToRemove)

                    if remainingOrders > 0:
                        newNode = orderNode(numberOfOrdersPlaced, -1, typeOfOrder)
                        newNode.numberOfStocks = remainingOrders
                        if stockName not in self.sellOrders:
                            self.sellOrders[stockName] = DoublyLinkedList()
                            self.sellOrders[stockName].insertAtPosition(1, newNode)
                        else:
                            newNode.numberOfStocks = remainingOrders
                            self.insertAtCorrectSellLMTPosition(newNode)
                    else:
                        self.completedOrders[stockName].append([typeOfOrder, "SELL", buyingPrice, numberOfOrdersPlaced])
        elif typeOfOrder == "LMT":
            askingPrice = self.determineBuyingPrice(userInput) # its the same as buying price 

            newNode = orderNode(numberOfOrdersPlaced, askingPrice, typeOfOrder)
            if stockName not in self.buyOrders:
                if stockName not in self.sellOrders:
                    self.sellOrders[stockName] = DoublyLinkedList()
                    self.sellOrders[stockName].insertAtPosition(1, newNode)
                else:
                    self.insertAtCorrectSellLMTPosition(newNode, stockName)
            
            else: # try to resolve the order 
                # print("======== starting here=====")
                startOfBuyOrder = self.buyOrders[stockName].head
                buyingPrice = startOfBuyOrder.price 
                remainingOrders = numberOfOrdersPlaced

                while remainingOrders > 0 and startOfBuyOrder:
                    # print(remainingOrders)
                    if (startOfBuyOrder.numberOfStocks >= remainingOrders and startOfBuyOrder.price == -1) or (startOfBuyOrder.numberOfStocks >= remainingOrders and startOfBuyOrder.price >= askingPrice) :
                        # print("Triggered")
                        initial = remainingOrders
                        remainingOrders -= startOfBuyOrder.numberOfStocks
                        startOfBuyOrder.numberOfStocks -= initial
                        
                        if stockName in self.completedOrders:
                            self.completedOrders[stockName].append(["LMT", "SELL", askingPrice, numberOfOrdersPlaced])
                            
                        else:
                            self.completedOrders[stockName] = [["LMT", "SELL", askingPrice, numberOfOrdersPlaced]]

                        if startOfBuyOrder.numberOfStocks == 0:
                            buyOrderType = "LMT"
                            # print("I love bts xoxo ")
                            if startOfBuyOrder.price == -1:
                                buyOrderType = "MKT"
                                # print("hello")
                            self.completedOrders[stockName].append([buyOrderType, "BUY", askingPrice, startOfBuyOrder.initialNumberOfOrders])
                            # print("here3")
                            self.buyOrders[stockName].remove(startOfBuyOrder)
                        startOfBuyOrder = startOfBuyOrder.next
                        
                        
                    elif (startOfBuyOrder.numberOfStocks < remainingOrders and startOfBuyOrder.price == -1) or (startOfBuyOrder.numberOfStocks < remainingOrders and startOfBuyOrder.price >= askingPrice):
                        # print("Should run 2 times")
                        nodeToRemove = startOfBuyOrder
                        remainingOrders -= nodeToRemove.numberOfStocks
                        if stockName in self.completedOrders:
                            self.completedOrders[stockName].append([nodeToRemove.typeOfOrder, "BUY", askingPrice, nodeToRemove.initialNumberOfOrders])
                        else:
                            self.completedOrders[stockName] = [[nodeToRemove.typeOfOrder, "BUY", askingPrice, nodeToRemove.initialNumberOfOrders]]
                        startOfBuyOrder = startOfBuyOrder.next
                        self.buyOrders[stockName].remove(nodeToRemove)
                    else:
                        startOfBuyOrder = startOfBuyOrder.next
                    
                    

                if remainingOrders > 0:
                    # print("generated")
                    newNode.numberOfStocks = remainingOrders
                    self.insertAtCorrectSellLMTPosition(newNode, stockName)



                



        
        return 

    def viewOrders(self):
        # O(m * n) time and O(1) space 
        # where m is the number of stocks and n is the number of orders on a stock.
        position = 1
        for stock in self.completedOrders:
            for completedOrder in self.completedOrders[stock]:
                typeOfOrder, action, price, amount = completedOrder
                print("{position}. {stock} {typeOfOrder} {action} ${price} {amount}/{amount} FILLED".format(position = position,stock = stock ,typeOfOrder = typeOfOrder, action = action, price = "{:.2f}".format(price), amount = amount))
                
                position += 1 
        for stock in self.buyOrders.keys():
            currentPtr = self.buyOrders[stock].head
            while currentPtr:
                typeOfOrder, action, price, remaining, amount = currentPtr.typeOfOrder, "BUY", currentPtr.price, currentPtr.initialNumberOfOrders - currentPtr.numberOfStocks, currentPtr.initialNumberOfOrders
                if remaining == 0:
                    if price == -1:
                        print("{position}. {stock} {typeOfOrder} {action} {remaining}/{amount} PENDING".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, remaining = remaining, amount = amount))
                    else:
                        
                        print("{position}. {stock} {typeOfOrder} {action} ${price} {remaining}/{amount} PENDING".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, price = "{:.2f}".format(price), remaining = remaining, amount = amount))
                else :
                    if price == -1:
                        print("{position}. {stock} {typeOfOrder} {action} {remaining}/{amount} PENDING".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, remaining = remaining, amount = amount))
                    else:
                        
                        print("{position}. {stock} {typeOfOrder} {action} ${price} {remaining}/{amount} PARTIAL".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, price = "{:.2f}".format(price), remaining = remaining, amount = amount))
                currentPtr = currentPtr.next 
                position += 1 
        
        for stock in self.sellOrders.keys():
            currentPtr = self.sellOrders[stock].head
            while currentPtr:
                typeOfOrder, action, price, remaining, amount = currentPtr.typeOfOrder, "SELL", currentPtr.price, currentPtr.initialNumberOfOrders - currentPtr.numberOfStocks, currentPtr.initialNumberOfOrders
                if remaining == 0:
                    if price == -1:
                        print("{position}. {stock} {typeOfOrder} {action} {remaining}/{amount} PENDING".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, remaining = remaining, amount = amount))
                    else:
                        
                        print("{position}. {stock} {typeOfOrder} {action} ${price} {remaining}/{amount} PENDING".format(position = position, stock = stock, typeOfOrder = typeOfOrder, action = action, price = "{:.2f}".format(price), remaining = remaining, amount = amount))
                else:
                    if price == -1:
                        print("{position}. {stock} {typeOfOrder} {action} {remaining}/{amount} PENDING".format(position = position, stock = stock , typeOfOrder = typeOfOrder, action = action, remaining = remaining, amount = amount))
                    else:
                        
                        print("{position}. {stock} {typeOfOrder} {action} ${price} {remaining}/{amount} PARTIAL".format(position = position, stock = stock, typeOfOrder = typeOfOrder, action = action, price = "{:.2f}".format(price), remaining = remaining, amount = amount))

                currentPtr = currentPtr.next 
                position += 1 


        return 

    def getLatestQuotes(self, userInput):
        #O(1) time | O(1) space 
        # well, string concatenation is by right O(n**2) but this is just simplifying the scenario, hence O(1) 
        stockName = userInput[2]
        if stockName in self.buyOrders and self.buyOrders[stockName].head:
            bid = self.buyOrders[stockName].tail.price
            bidStatement = "BID:" +  "{price}".format(price = bid)
        else:
            bidStatement = "BID:" + "No current bid prices yet!"

        if stockName in self.sellOrders and self.sellOrders[stockName].head:
            ask = self.sellOrders[stockName].tail.price
            askStatement = "ASK:" + "{ask}".format(ask = ask)
        else:
            askStatement = "ASK:" + "No current asking prices yet!"
        
        if stockName in self.completedOrders:
            last = self.completedOrders[stockName][-1][2]
            lastStatement = "LAST:" + "{last}".format(last = last)
        else:
            lastStatement = "LAST:" + "Item has not been sold before!"



        return stockName + " " + bidStatement + " " + askStatement + " " + lastStatement

    def determineTypeOfOrder(self, userInput):
        if len(userInput) < 5:
            return None
        elif userInput[3] == "MKT":
            return "MKT"
        elif userInput[3] == "LMT":
            return "LMT"
        
        return None

    def determineStockName(self, userInput):
        if len(userInput) < 3:
            return None
        else:
            return userInput[2]

    def determineNumbersPlaced(self, typeOfOrder, userInput):
        if typeOfOrder == "LMT" and len(userInput) == 6:

            try:
                numberOfOrder = int(userInput[5])

                return numberOfOrder
            except:
                print("numberOfOrder is not an integer!")

            
        elif typeOfOrder == "MKT" and len(userInput) == 5:
            try:
                numberOfOrder = int(userInput[4])

                return numberOfOrder
            except:
                print("numberOfOrder is not an integer!")

        else:
            print("invalidNumbersPlaced")

            return
    def determineBuyingPrice(self, userInput):
        if len(userInput) != 6:
            print("invalid Input, determine buying price")
            return 
        toReturn = userInput[4][1:]
        return float(toReturn)
            
    def insertAtCorrectBuyLMTPosition(self, node, stockName):
        if stockName not in self.buyOrders:
            self.buyOrders[stockName] = DoublyLinkedList()
            self.buyOrders[stockName].insertAtPosition(1, node)
        currentPtr = node 
        startingNode = self.buyOrders[stockName].head
        position = 1 
        # print(currentPtr.price, startingNode.price)
        while startingNode != None and currentPtr.price >= startingNode.price:
            # print("here")
            startingNode = startingNode.next
            position += 1 
        self.buyOrders[stockName].insertAtPosition(position, node)
        # print("inserted at position", position)
        return 

    def insertAtCorrectSellLMTPosition(self, node, stockName):
        if stockName not in self.sellOrders:
            self.sellOrders[stockName] = DoublyLinkedList()
            self.sellOrders[stockName].insertAtPosition(1, node)
        currentPtr = node 
        startingNode = self.sellOrders[stockName].head
        position = 1 
        while startingNode != None and currentPtr.price >= startingNode.price:
            startingNode = startingNode.next
            position += 1 
        self.sellOrders[stockName].insertAtPosition(position, node)
        # print("inserted at position", position)
        return 



    
    

# this is an order node 
class orderNode:
    def __init__(self, numberOfStocks, price, typeOfOrder):
        self.numberOfStocks = int(numberOfStocks) 
        self.initialNumberOfOrders = int(numberOfStocks)
        self.price = price
        self.typeOfOrder = typeOfOrder
        self.prev = None 
        self.next = None

# this is a deque (could just use the library but I think I want more custom methods)

class DoublyLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def setHead(self, node):

        # O(1) operation 
        if self.head is None:
            self.head = node 
            self.tail = node 
            
            return 
        self.insertBefore(self.head, node)
        
        return 

    def setTail(self, node):

        # O(1) operation 
        if self.tail is None:
            self.head = node 
            self.tail = node 
            return 
        self.insertAfter(self.tail, node)
        return 
        

    def insertBefore(self, node, nodeToInsert):

        if nodeToInsert == self.tail and nodeToInsert == self.head:
            return 
        
        self.remove(nodeToInsert)
        nodeToInsert.prev = node.prev 
        nodeToInsert.next = node 
        if node.prev is None:
            self.head = nodeToInsert
        else:
            node.prev.next = nodeToInsert
        node.prev = nodeToInsert
        
        

    def insertAfter(self, node, nodeToInsert):

        if nodeToInsert == self.head and nodeToInsert == self.tail:
            return
        
        self.remove(nodeToInsert)
        nodeToInsert.prev  = node 
        nodeToInsert.next = node.next 
        
        if node == self.tail:
            self.tail = nodeToInsert
            nodeToInsert.next = None
            self.tail = nodeToInsert 
        else:
            node.next.prev = nodeToInsert

            
        node.next = nodeToInsert
            
        return 

    def insertAtPosition(self, position, nodeToInsert):

        if position == 1:
            self.setHead(nodeToInsert)
            return 
        
        node = self.head 
        currentPosition = 1 
        while node is not None and currentPosition != position:
            node = node.next 
            currentPosition += 1 
        if node is not None:
            self.insertBefore(node, nodeToInsert)
        else:
            self.setTail(nodeToInsert)
        return


    def removeNodesWithValue(self, value):

        node = self.head 
        while node is not None:
            nodeToRemove = node 
            node = node.next 
            if nodeToRemove.value == value:
                self.remove(nodeToRemove)
        return 

    def remove(self, node):

        if node == self.head:
            self.head = self.head.next 
        if node == self.tail:
            self.tail = self.tail.prev
        self.removeNodeBindings(node)
        pass

    def containsNodeWithValue(self, value):

        node = self.head 
        while node is not None and node.value != value:
            node = node.next 
        return node is not None

    def removeNodeBindings(self, node):
        if node.prev is not None:
            node.prev.next = node.next 
        if node.next is not None:
            node.next.prev = node.prev
        node.prev = None
        node.next = None



In [10]:
# get user input first 
# buy 
# sell 
# bid / ask for last price 
# view all orderstatus 
# exit the program 

userInput = input()

#initialise stock market object 

stockExchange = stockMarket()

while userInput != "Action: QUIT":
    # there are 3 actions: buy view and sell 
    userInputSplitted = userInput.split()
#     print(userInputSplitted)
    # since its guranteed Action is going to be the first word. 
    # check for validity of the input (length > 1)
    if len(userInputSplitted) <= 1:
        print("INVALID INPUT!")
        userInput = input()
        continue
    
    # check which action it is
    # USER INPUT :
    #buy / sell + Limit Order [Action, buy/sell/view, stockName, typeOfOrder, price, quantity]
    #buy / sell + Market Order [Action, buy/sell/view, stockName, typeOfOrder, quantity]
    #view [Action, view, orders]
    #QUOTE [Action, Quote, stockName]


    typeOfAction = userInputSplitted[1].upper() # make sure its caps
    # this is the buy action
    if typeOfAction == "BUY":
        # remember market vs limit order 
#         print("Buy order")
        stockExchange.buyOrder(userInputSplitted)
        typeOfOrder = userInputSplitted[3]
        if typeOfOrder == "LMT":
            numberOfOrdersPlaced = userInputSplitted[5]
            buyingPrice = userInputSplitted[4]
            stockName = userInputSplitted[2]
            returnTypeStatement = "limit buy order"
            print("You have placed a {returnTypeStatement} for {numberOfOrdersPlaced} {stockName} shares at {buyingPrice} each".format(returnTypeStatement = returnTypeStatement, numberOfOrdersPlaced = numberOfOrdersPlaced, stockName = stockName, buyingPrice = buyingPrice))
        else:
            numberOfOrdersPlaced = userInputSplitted[4]
            stockName = userInputSplitted[2]
            returnTypeStatement = "marker order"
            print("You have placed a {returnTypeStatement} for {numberOfOrdersPlaced} {stockName} shares".format(returnTypeStatement = returnTypeStatement, numberOfOrdersPlaced = numberOfOrdersPlaced, stockName = stockName))
    
    # this is the sell action 
    elif typeOfAction == "SELL":
        # remember market vs limit order
        stockExchange.sellOrder(userInputSplitted)
        typeOfOrder = userInputSplitted[3]
        if typeOfOrder == "LMT":
            numberOfOrdersPlaced = userInputSplitted[5]
            buyingPrice = userInputSplitted[4]
            stockName = userInputSplitted[2]
            returnTypeStatement = "limit sell order"
            print("You have placed a {returnTypeStatement} for {numberOfOrdersPlaced} {stockName} shares at {buyingPrice} each".format(returnTypeStatement = returnTypeStatement, numberOfOrdersPlaced = numberOfOrdersPlaced, stockName = stockName, buyingPrice = buyingPrice))
        else:
            returnTypeStatement = "marker order"
            numberOfOrdersPlaced = userInputSplitted[4]
            stockName = userInputSplitted[2]
            print("You have placed a {returnTypeStatement} for {numberOfOrdersPlaced} {stockName} shares".format(returnTypeStatement = returnTypeStatement, numberOfOrdersPlaced = numberOfOrdersPlaced, stockName = stockName))
    
    # retrieve action
    elif typeOfAction == "VIEW":
        stockExchange.viewOrders()


    elif typeOfAction == "QUOTE":
        quote = stockExchange.getLatestQuotes(userInputSplitted)
        print(quote)
    else:
        print("invalid Input! input")
        



    
    userInput = input()

Action: BUY SNAP LMT $30 100
You have placed a limit buy order for 100 SNAP shares at $30 each
Action: VIEW ORDERS
1. SNAP LMT BUY $30.00 0/100 PENDING
Action: BUY FB MKT 20 
You have placed a marker order for 20 FB shares
Action: VIEW ORDERS
1. SNAP LMT BUY $30.00 0/100 PENDING
2. FB MKT BUY 0/20 PENDING
Action: SELL FB LMT $20.00 20
You have placed a limit sell order for 20 FB shares at $20.00 each
Action: VIEW ORDERS
1. FB LMT SELL $20.00 20/20 FILLED
2. FB MKT BUY $20.00 20/20 FILLED
3. SNAP LMT BUY $30.00 0/100 PENDING
Action: SELL SNAP LMT $30.00 20 
You have placed a limit sell order for 20 SNAP shares at $30.00 each
Action: VIEW ORDERS
1. FB LMT SELL $20.00 20/20 FILLED
2. FB MKT BUY $20.00 20/20 FILLED
3. SNAP LMT SELL $30.00 20/20 FILLED
4. SNAP LMT BUY $30.00 20/100 PARTIAL
Action: SELL SNAP LMT $31.00 10
You have placed a limit sell order for 10 SNAP shares at $31.00 each
Action: QUOTE SNAP
SNAP BID:30.0 ASK:31.0 LAST:30.0
Action: QUIT
